In [18]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [19]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-0t165v3c
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-0t165v3c


In [20]:
!pip install kss

In [21]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, notebook

import math
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

import pandas as pd

#json 관련 라이브러리
import json
from collections import OrderedDict

#문장 분리기
from kss import split_sentences


In [22]:
#구글 드라이브를 쓰자
from google.colab import drive
#루트를 정해준다.
ROOT = "/content/drive"
print(ROOT)

drive.mount(ROOT) #we mount the google drive at /content/drive

#import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join


MY_GOOGLE_DRIVE_PATH = '/content/drive/My Drive/Colab Notebooks'

print("MY_GOOGLE_DRIVE_PATH: ", MY_GOOGLE_DRIVE_PATH)

%cd "{MY_GOOGLE_DRIVE_PATH}"

/content/drive
Mounted at /content/drive
MY_GOOGLE_DRIVE_PATH:  /content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks


In [23]:
device = torch.device("cuda:0")

In [24]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [25]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [26]:
max_len = 200
batch_size = 32

In [27]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [28]:
situation_model = torch.load("situation7.pth")
emotion_model = torch.load("model4.pth")

In [42]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict_emotion(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size)
    
    emotion_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = emotion_model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("분노")
            elif np.argmax(logits) == 1:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 2:
                test_eval.append("불안")
            elif np.argmax(logits) == 3:
                test_eval.append("상처")
            elif np.argmax(logits) == 4:
                test_eval.append("당황")
            elif np.argmax(logits) == 5:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 6:
                test_eval.append("중립")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
        return test_eval[0]


def predict_situation(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size)
    
    situation_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = situation_model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("가족관계")
            elif np.argmax(logits) == 1:
                test_eval.append("학업 및 진로")
            elif np.argmax(logits) == 2:
                test_eval.append("학교폭력/따돌림")
            elif np.argmax(logits) == 3:
                test_eval.append("대인관계")
            elif np.argmax(logits) == 4:
                test_eval.append("직장, 업무 스트레스")
            elif np.argmax(logits) == 5:
                test_eval.append("연애, 결혼, 출산")
            elif np.argmax(logits) == 6:
                test_eval.append("진로, 취업, 직장")
            elif np.argmax(logits) == 7:
                test_eval.append("재정, 은퇴, 노후준비")
            elif np.argmax(logits) == 8:
                test_eval.append("건강, 죽음")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 상황입니다.")

        return test_eval[0]

using cached model


In [193]:
##문장 넣기
setences = []
text = ""
text = input("하고싶은 말을 입력해주세요 : ")

setences= split_sentences(text)

print(setences, '\n')

emotion_list= []
situation_list = []
for i, sent in enumerate(setences) :
  emotion_data = []
  emotion_pre = predict_emotion(sent)
  emotion_data.append(sent)
  emotion_data.append(str(emotion_pre))
  emotion_list.append(emotion_data)


  situation_data = []
  situation_pre = predict_situation(sent)
  situation_data.append(sent)
  situation_data.append(str(situation_pre))
  situation_list.append(situation_data)



하고싶은 말을 입력해주세요 : 지난 일주일. 나의 출퇴근 시간과 짬시간들을 채워준 책이었다. 베를린에서 머문 90일간의 경험과 생각을 담아 쓴 일기. 참 유쾌하고 엉뚱 발랄한 책이다. 어쩌면 그리 특별하지 않은 소소한 일상이었을 수도 있는데(하긴 독일에서 체류한다는 것 자체가 특별하긴 하지) 그 속에서 말할 꺼리를 찾아내고 그걸 또 맛깔나게 써내려가는 능력이 대단하다 싶었다. 그런 게 글쟁이들의 일이고 실력이겠지만 여튼… 중요한 건 읽는 내내 혼자 큭큭거리다 빵터지다를 반복할만큼 재미있었다는 것. 남의 일기를 훔쳐보는 듯한 관음증적인 심리에 더해져서일까? 술에 취해 맞춤법도 틀리고 횡설수설 써내려간  써내려갔던 11월 4일의 일기가 기억에 남는다.  그리고 다닐로(11월 11일자 일기)를 비롯해 여타 다른 친구들에게 쓴 마음의(차마 말로는 못하는) 편지들 ㅋㅋㅋ 독일어를 배우던 중 (엉뚱하게도) 스페인어를 배워야겠다는 깨달음을 얻으면서 동시에 영어가 왜 그렇게 중요한지에 대한 이유를 설명하던 12월 3일자 일기에도 큰 공감이 갔다. 이 책을 읽으며 내용도 내용이지만 새로운 글쓰기 기법을 만들어내는 작가의 능력에 대해서도 생각하게 되었다.  일기라는 것을 보여주려는 듯 "이 글은 ***** 를 하며 쓰고 있다"처럼 시작하는 것도 현장감(?)을 더해줬고, 그냥 "그저 내 생각일 뿐이야"라는 듯 "***라는 인상을 준다"라는 말투를 반복해서 쓴 것도 나름 재치있었다. "이런 내 기분은 ***를 했던 *** 만이 알아줄 것이다"는 표현도 참 적절했다는 생각이 든다. 스산했을 그곳에서 느꼈을 외로움이 흠씬 느껴지는 듯한 표현^^ ​ 다 읽고 나서 왠지 모를 아쉬움이 남았지만 남의 일기를 두 번 읽는 것은 왠지 스토커가 되는 기분이 들어 같아 그만두기로 했다. 대신 작가분의 블로그에 들어가 "이웃 추가" 버튼을 지긋이 눌렀다. 가끔 웃음이 필요하다 싶을 때 들어가 보려고~


IndexError: ignored

In [183]:
emotion_list

[['진짜 임시저장한지 한 달 다되가믄데 오늘 아침 출근길 온도 보고 이건 더는 늦으면 안되겠어서 씀니다', '중립'],
 ['수연맘 예설맘 과 함께한 여름일기 지난 프리뷰 나 머리 짜르러 또 가야 하는데 머리 첨에 하면 넘 예뿐데 내가 똥손이라 결말은 항상 신돈 수백번의 시행착오를 겪고 난뒤의 간촐한 여행짐 우리의 이번 피크닉 장소는 강릉강릉 그래서 언제! 나는 따로 출발해서 먼더 도착해서 예슬수연이 기다렸다',
  '기쁨'],
 ['우린 뚜벅초 들이라 모두 택시로 이동 숙소로 가는길이 전부 솔밭이라 산림욕과 해수욕이 쌉가능 이번 여행 다 좋았디만 숙소가 제일 좋았음 나 에어비앤비 예약 첨해봐짜냐.',
  '중립'],
 ['송정바다 찢으러 왓다^^ 내가 수연이 한테 수영복 사진 들이라 나 감성비취 할려고 수건도 사짜냐 나 이번 강릉 바다에서 네시호 괴물 봣쟈냐 여기 수심 2M 인데 예슬이 키가 183이긴 하지만 그래도 대단한 깊이ㅠㅠ since 2004 ^^ 이뒤로 메이크업 할 생각이 없었기때메 안구보호차원 필터',
  '중립']]

In [184]:
situation_list

[['진짜 임시저장한지 한 달 다되가믄데 오늘 아침 출근길 온도 보고 이건 더는 늦으면 안되겠어서 씀니다', '건강, 죽음'],
 ['수연맘 예설맘 과 함께한 여름일기 지난 프리뷰 나 머리 짜르러 또 가야 하는데 머리 첨에 하면 넘 예뿐데 내가 똥손이라 결말은 항상 신돈 수백번의 시행착오를 겪고 난뒤의 간촐한 여행짐 우리의 이번 피크닉 장소는 강릉강릉 그래서 언제! 나는 따로 출발해서 먼더 도착해서 예슬수연이 기다렸다',
  '대인관계'],
 ['우린 뚜벅초 들이라 모두 택시로 이동 숙소로 가는길이 전부 솔밭이라 산림욕과 해수욕이 쌉가능 이번 여행 다 좋았디만 숙소가 제일 좋았음 나 에어비앤비 예약 첨해봐짜냐.',
  '대인관계'],
 ['송정바다 찢으러 왓다^^ 내가 수연이 한테 수영복 사진 들이라 나 감성비취 할려고 수건도 사짜냐 나 이번 강릉 바다에서 네시호 괴물 봣쟈냐 여기 수심 2M 인데 예슬이 키가 183이긴 하지만 그래도 대단한 깊이ㅠㅠ since 2004 ^^ 이뒤로 메이크업 할 생각이 없었기때메 안구보호차원 필터',
  '건강, 죽음']]

In [185]:
cnt_total1=0
cnt1=[0,0,0,0,0,0,0]

for i in range(len(emotion_list)):
    cnt_total1+=1
    if emotion_list[i][1]=='분노':
        cnt1[0]+=1
    elif emotion_list[i][1]=='슬픔':
        cnt1[1]+=1
    elif emotion_list[i][1]=='불안':
        cnt1[2]+=1
    elif emotion_list[i][1]=='상처':
        cnt1[3]+=1
    elif emotion_list[i][1]=='당황':
        cnt1[4]+=1
    elif emotion_list[i][1]=='기쁨':
        cnt1[5]+=1
    elif emotion_list[i][1]=='중립':
        cnt1[6]+=1
    else:
        print('error')
    
emo_per=round(max(cnt1)/cnt_total1,4)

In [186]:
emotions_dict={0:'분노',1:'슬픔',2:'불안',3:'상처',4:'당황',5:'기쁨', 6:"중립"}

situations_dict={0:'가족관계',
1:'학업 및 진로',
2:'학교폭력/따돌림',
3:'대인관계',
4:'직장, 업무 스트레스',
5:'연애, 결혼, 출산',
6:'진로, 취업, 직장',
7:'재정, 은퇴, 노후준비',
8:'건강, 죽음',
}

In [187]:

cnt_total2=0
cnt2=[0,0,0,0,0,0,0,0,0]

for j in range(len(situation_list)):
    cnt_total2+=1
    if situation_list[j][1]=='가족관계':
        cnt2[0]+=1
    elif situation_list[j][1]=='학업 및 진로':
        cnt2[1]+=1
    elif situation_list[j][1]=='학교폭력/따돌림':
        cnt2[2]+=1
    elif situation_list[j][1]=='대인관계':
        cnt2[3]+=1
    elif situation_list[j][1]=='직장, 업무 스트레스':
        cnt2[4]+=1
    elif situation_list[j][1]=='연애, 결혼, 출산':
        cnt2[5]+=1
    elif situation_list[j][1]=='진로, 취업, 직장':
        cnt2[6]+=1
    elif situation_list[j][1]=='재정, 은퇴, 노후준비':
        cnt2[7]+=1
    elif situation_list[j][1]=='건강, 죽음':
        cnt2[8]+=1
    else:
        print('error')


In [188]:
obj={
    'emotion':{"분노":round(cnt1[0]/cnt_total1*100,4),
            "슬픔":round(cnt1[1]/cnt_total1*100,4),
            "불안":round(cnt1[2]/cnt_total1*100,4),
            "상처":round(cnt1[3]/cnt_total1*100,4),
            "당황":round(cnt1[4]/cnt_total1*100,4),
            "기쁨":round(cnt1[5]/cnt_total1*100,4),
            "중립":round(cnt1[6]/cnt_total1*100,4)},
  
'sentence':{"분노":[],
            "슬픔":[],
            "불안":[],
            "상처":[],
            "당황":[],
            "기쁨":[],
            "중립":[]},
'situation':{emotions_dict[cnt1.index(max(cnt1))]:situations_dict[cnt2.index(max(cnt2))]}
}


In [189]:


for i in range(len(emotion_list)):
    cnt_total1+=1
    if emotion_list[i][1]=='분노':
        obj['sentence']['분노']=emotion_list[i][0]
    elif emotion_list[i][1]=='슬픔':
        obj['sentence']['슬픔'].append(emotion_list[i][0])
    elif emotion_list[i][1]=='불안':
        obj['sentence']['불안'].append(emotion_list[i][0])
    elif emotion_list[i][1]=='상처':
        obj['sentence']['상처'].append(emotion_list[i][0])
    elif emotion_list[i][1]=='당황':
        obj['sentence']['당황'].append(emotion_list[i][0])
    elif emotion_list[i][1]=='기쁨':
        obj['sentence']['기쁨'].append(emotion_list[i][0])
    elif emotion_list[i][1]=='중립':
        obj['sentence']['중립'].append(emotion_list[i][0])
    else:
        print('error')
    
emo_per=round(max(cnt1)/cnt_total1,4)

In [190]:
obj

{'emotion': {'기쁨': 25.0,
  '당황': 0.0,
  '분노': 0.0,
  '불안': 0.0,
  '상처': 0.0,
  '슬픔': 0.0,
  '중립': 75.0},
 'sentence': {'기쁨': ['수연맘 예설맘 과 함께한 여름일기 지난 프리뷰 나 머리 짜르러 또 가야 하는데 머리 첨에 하면 넘 예뿐데 내가 똥손이라 결말은 항상 신돈 수백번의 시행착오를 겪고 난뒤의 간촐한 여행짐 우리의 이번 피크닉 장소는 강릉강릉 그래서 언제! 나는 따로 출발해서 먼더 도착해서 예슬수연이 기다렸다'],
  '당황': [],
  '분노': [],
  '불안': [],
  '상처': [],
  '슬픔': [],
  '중립': ['진짜 임시저장한지 한 달 다되가믄데 오늘 아침 출근길 온도 보고 이건 더는 늦으면 안되겠어서 씀니다',
   '우린 뚜벅초 들이라 모두 택시로 이동 숙소로 가는길이 전부 솔밭이라 산림욕과 해수욕이 쌉가능 이번 여행 다 좋았디만 숙소가 제일 좋았음 나 에어비앤비 예약 첨해봐짜냐.',
   '송정바다 찢으러 왓다^^ 내가 수연이 한테 수영복 사진 들이라 나 감성비취 할려고 수건도 사짜냐 나 이번 강릉 바다에서 네시호 괴물 봣쟈냐 여기 수심 2M 인데 예슬이 키가 183이긴 하지만 그래도 대단한 깊이ㅠㅠ since 2004 ^^ 이뒤로 메이크업 할 생각이 없었기때메 안구보호차원 필터']},
 'situation': {'중립': '대인관계'}}

In [118]:
with open("output to json", "w") as new_file:
	json.dump(obj, new_file,indent=4, sort_keys=True)